In [1]:
# https://dacon.io/competitions/official/235930/codeshare/5508?page=1&dtype=recent


In [2]:
import random
import pandas as pd
import numpy as np
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cuda')

In [5]:
epochs = 100
lr = 1e-3
batch_size = 512
momentum = 0.9
weight_decay = 1e-4
seed = 42
NUM_WORKERS = 8
saved_model = '../saved/ae_embeding/best_model.pth'

param = {
            'epochs' : epochs,
            'lr' :lr,
            'batch_size' : batch_size,
            'momentum' : momentum,
            'weight_decay' : weight_decay
        }

In [6]:
def seed_everything(seed) :
    random.seed(seed)
    os.environ['PYHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed)    

In [7]:
train_df = pd.read_csv('../dataset/train.csv')
val_df = pd.read_csv('../dataset/val.csv')
train_df = train_df.drop(columns=['ID'])
val_df = val_df.drop(columns=['ID'])
print(train_df.shape)

(113842, 30)


In [8]:
# class CDataset(Dataset) :
#     def __init__(self, df, eval_mode=False) :
#         self.df = df
#         self.eval_mode = eval_mode
#         if self.eval_mode :
#             self.labels = self.df['Class']
#             self.df = self.df.drop(columns=['Class'])
        
#     def __getitem__(self, idx) :
#         if self.eval_mode :
#             self.x = self.df.iloc[idx, 1:]
#             self.y = self.labels[idx]
#             return torch.tensor(self.x, dtype=torch.float32),torch.tensor(self.y)
        
#         else :
#             self.x = self.df.iloc[idx, 1:]
#             return torch.tensor(self.x, dtype=torch.float32)
        
#     def __len__(self) :
#         return len(self.df)
        

In [9]:
class CDataset(Dataset):
    def __init__(self, df, eval_mode=False):
        self.df = df
        self.eval_mode = eval_mode
        if self.eval_mode:
            self.labels = self.df['Class'].values
            self.df = self.df.drop(columns=['Class']).values
        else:
            self.df = self.df.values
        
    def __getitem__(self, index):
        if self.eval_mode:
            self.x = self.df[index]
            self.y = self.labels[index]
            return torch.Tensor(self.x), self.y
        else:
            self.x = self.df[index]
            return torch.Tensor(self.x)
        
    def __len__(self):
        return len(self.df)

In [10]:
train_dataset = CDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True, num_workers=NUM_WORKERS)

val_dataset = CDataset(val_df, eval_mode=True)
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle=False, num_workers=NUM_WORKERS)

In [11]:
x = next(iter(train_loader))
x.shape

torch.Size([512, 30])

In [12]:
x, y  = next(iter(val_loader))
x.shape

torch.Size([512, 30])

In [13]:
class AutoEncoder(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.dim = 30
        self.embeding_dim = 1024
        self.hidden = 256
        self.act = nn.GELU()
        
        self.embeding = nn.Sequential(
            nn.Linear(self.dim,self.hidden),
            nn.GELU(),
            nn.Linear(self.hidden,self.embeding_dim),
            nn.GELU(),
            # self.act
        )
        self.encoder = nn.Sequential(
            # nn.BatchNorm1d(self.embeding_dim),
            nn.Linear(self.embeding_dim,self.hidden),
            nn.BatchNorm1d(self.hidden),
            self.act,
            # nn.Linear(64,128),
            # nn.BatchNorm1d(128),
            # nn.LeakyReLU(),
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(self.hidden,self.embeding_dim),
            nn.BatchNorm1d(self.embeding_dim),
            self.act,
            # nn.Linear(64,30),
        )
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)  
                
        
#         freezing embeding layer        
        for name, child in self.named_children() :
            for param in child.parameters() :
                if name == 'embeding' :
                    param.requires_grad = False                
        
    def forward(self, x) :
        x_ = self.embeding(x)
        x = self.encoder(x_)
        x = self.decoder(x)
        return x_, x
        

In [14]:
class Trainer() :
    def __init__(self, model, optimizer, train_loader, val_loader, scheduler, device, **param) :
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.scheduler = scheduler
        self.device = device
        self.batch_size = param['batch_size']
        self.epochs = param['epochs']
        self.lr = param['lr']
        
        self.criterion = nn.L1Loss().to(device)
        
    def fit(self,) :
        self.model.to(self.device)
        best_score = 0
        for epoch in range(self.epochs) :
            self.model.train()
            train_loss = []
            
            for x in iter(self.train_loader) :
                x = x.to(self.device)
                x_, x = self.model(x)
                
                loss = self.criterion(x_, x)
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
                train_loss.append(loss.item())
                # self.scheduler.step()
                
            score = self.validation(self.model, 0.95)
            
            if self.scheduler is not None :
                self.scheduler.step(score)
            
            print(f'epoch :[{epoch}] train loss [{np.mean(train_loss)}] val score [{score}]')
            for param_group in self.optimizer.param_groups:
                print(param_group['lr'])      
            
            # print(f'epoch :[{epoch}] train loss [{np.mean(train_loss)}] val score [{score}] lr [{self.scheduler.get_lr()}]')

            if best_score < score :
                best_score = score
                torch.save(self.model.state_dict(), saved_model)
            
    def validation(self, eval_model, threshold) :
        cos = nn.CosineSimilarity(dim=1, eps=1e-6)
        eval_model.eval()
        pred_y = []
        true_y = []
        
        with torch.no_grad():
            for x, y in iter(self.val_loader) :
                x = x.to(self.device)
                y = y.to(self.device)
                
                x_, x = self.model(x)
                diff = cos(x, x_).cpu().tolist()
                batch_pred = np.where(np.array(diff) < threshold, 1, 0).tolist()
                pred_y += batch_pred
                true_y += y.tolist()
                
        return f1_score(true_y, pred_y, average='macro')

In [15]:
model = AutoEncoder()

In [16]:
model = AutoEncoder()
model.eval()
optimizer = torch.optim.Adam(params=model.parameters(), lr = param['lr'])

# optimizer = torch.optim.SGD(model.parameters(), param['lr'],
#                             momentum=param['momentum'],
#                             weight_decay=param['weight_decay'])
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)
# scheduler = StepLR(optimizer, step_size=50, gamma=0.2)
scheduler = None

# scheduler = lr_scheduler.LambdaLR(optimizer, lambda epoch : 0.1 **(epoch //30))
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 
#                                                 epochs              = param['epochs'], 
#                                                 steps_per_epoch     = int(len(train_dataset)/param['batch_size'])+1,
#                                                 max_lr              = param['lr'], 
#                                                 pct_start           = 0.05, 
#                                                 div_factor          = 5, 
#                                                 final_div_factor    = 5e+4)   


In [17]:
trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device, **param)
# trainer = Trainer(model, optimizer, train_loader, val_loader, scheduler, device)

In [18]:
trainer.fit()

epoch :[0] train loss [0.06956863662253046] val score [0.3616257679568677]
0.001
epoch :[1] train loss [0.03399452332390531] val score [0.46121662471959296]
0.001
epoch :[2] train loss [0.02862352868667365] val score [0.4912791125567069]
0.001
epoch :[3] train loss [0.02666492710900681] val score [0.5028884467292228]
0.001
epoch :[4] train loss [0.02513615147214834] val score [0.5002085946595167]
0.001
epoch :[5] train loss [0.023795358402312068] val score [0.5159622195226325]
0.001
epoch :[6] train loss [0.022328537817827256] val score [0.5267983031167234]
0.001
epoch :[7] train loss [0.022606853454169136] val score [0.5310717875219239]
0.001
epoch :[8] train loss [0.020951675877576452] val score [0.5277460002362148]
0.001
epoch :[9] train loss [0.019832260639410917] val score [0.5405656326020356]
0.001
epoch :[10] train loss [0.019163792932013493] val score [0.5499639140767333]
0.001
epoch :[11] train loss [0.01927045005636766] val score [0.5492980233335744]
0.001
epoch :[12] train l